In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/india-air-quality-data/data.csv


# Importing Libraries

In [2]:
import numpy as np  # linear algebra
import pandas as pd  # Data manipulation and analysis
import matplotlib.pyplot as plt # data visualization

# Importing CSV from Kaggle Air Quality dataset of India 

In [3]:
df =  pd.read_csv("/kaggle/input/india-air-quality-data/data.csv", encoding='ISO-8859-1', low_memory=False)
df.head()

,stn_code,sampling_date,state,location,agency,type,so2,no2,rspm,spm,location_monitoring_station,pm2_5,date
0,150,February - M021990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",4.8,17.4,NaN,NaN,NaN,NaN,1990-02-01
1,151,February - M021990,Andhra Pradesh,Hyderabad,NaN,Industrial Area,3.1,7.0,NaN,NaN,NaN,NaN,1990-02-01
2,152,February - M021990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",6.2,28.5,NaN,NaN,NaN,NaN,1990-02-01
3,150,March - M031990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",6.3,14.7,NaN,NaN,NaN,NaN,1990-03-01
4,151,March - M031990,Andhra Pradesh,Hyderabad,NaN,Industrial Area,4.7,7.5,NaN,NaN,NaN,NaN,1990-03-01


In [4]:
print(df.shape)
print(df.describe())

(435742, 13)
                 so2            no2           rspm            spm        pm2_5
count  401096.000000  419509.000000  395520.000000  198355.000000  9314.000000
mean       10.829414      25.809623     108.832784     220.783480    40.791467
std        11.177187      18.503086      74.872430     151.395457    30.832525
min         0.000000       0.000000       0.000000       0.000000     3.000000
25%         5.000000      14.000000      56.000000     111.000000    24.000000
50%         8.000000      22.000000      90.000000     187.000000    32.000000
75%        13.700000      32.200000     142.000000     296.000000    46.000000
max       909.000000     876.000000    6307.033333    3380.000000   504.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435742 entries, 0 to 435741
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   stn_code                     291665 non-null  object 
 1   sampling_date                435739 non-null  object 
 2   state                        435742 non-null  object 
 3   location                     435739 non-null  object 
 4   agency                       286261 non-null  object 
 5   type                         430349 non-null  object 
 6   so2                          401096 non-null  float64
 7   no2                          419509 non-null  float64
 8   rspm                         395520 non-null  float64
 9   spm                          198355 non-null  float64
 10  location_monitoring_station  408251 non-null  object 
 11  pm2_5                        9314 non-null    float64
 12  date                         435735 non-null  object 
dtyp

# Data Preprocessing

In [6]:
df.isnull().sum()

stn_code                       144077
sampling_date                       3
state                               0
location                            3
agency                         149481
type                             5393
so2                             34646
no2                             16233
rspm                            40222
spm                            237387
location_monitoring_station     27491
pm2_5                          426428
date                                7
dtype: int64

In [7]:
# Removing low cardinality and High cardinality columns
df.drop(columns=["stn_code", "agency", "pm2_5"], inplace=True)    

In [8]:
# leaky data
df.drop(columns=["sampling_date"], inplace=True)

## Deleting missing Value

In [9]:
df_clean = df.dropna()
df_clean.info()
df_clean.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140823 entries, 1036 to 431457
Data columns (total 9 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   state                        140823 non-null  object 
 1   location                     140823 non-null  object 
 2   type                         140823 non-null  object 
 3   so2                          140823 non-null  float64
 4   no2                          140823 non-null  float64
 5   rspm                         140823 non-null  float64
 6   spm                          140823 non-null  float64
 7   location_monitoring_station  140823 non-null  object 
 8   date                         140823 non-null  object 
dtypes: float64(4), object(5)
memory usage: 10.7+ MB


(140823, 9)

In [10]:
grouped_data = df.groupby('state')

In [11]:
highest_values = {}

# Iterate through each group and find the maximum value for each column
for state, group in grouped_data:
    highest_values[state] = {
        'so2': group['so2'].max(),
        'no2': group['no2'].max(),
        'rspm': group['rspm'].max(),
        'spm': group['spm'].max()
    }

# Display the highest values for each column in each state
df_highest_values = pd.DataFrame(highest_values)

# Transpose the DataFrame to have states as rows and columns as the columns of the original DataFrame
df_highest_values = df_highest_values.T
df_highest_values.index.name = 'state'

# Display the DataFrame
print(df_highest_values)


                                 so2     no2         rspm     spm
state                                                            
Andhra Pradesh               228.000  334.90   637.000000  1082.0
Arunachal Pradesh             13.000   17.00   250.000000     NaN
Assam                         56.000  147.90   790.000000  1300.0
Bihar                        216.600  203.80   656.000000  1338.0
Chandigarh                    55.500  103.00   678.000000  1752.0
Chhattisgarh                  29.300   92.00   477.000000   650.0
Dadra & Nagar Haveli          23.900   39.30   152.000000   390.0
Daman & Diu                   34.300   51.00   144.000000   330.0
Delhi                        127.400  224.00   892.000000  1885.0
Goa                          108.600   81.00   495.000000   736.0
Gujarat                      219.100  279.80   411.330000  1654.0
Haryana                       48.300  188.00  1082.000000  2213.0
Himachal Pradesh              81.000  696.00   852.000000  1816.0
Jammu & Ka

In [12]:
df_highest_values.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, Andhra Pradesh to andaman-and-nicobar-islands
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   so2     34 non-null     float64
 1   no2     34 non-null     float64
 2   rspm    34 non-null     float64
 3   spm     32 non-null     float64
dtypes: float64(4)
memory usage: 1.4+ KB


In [13]:
# Calculate average pollution level for each state
df['average_pollution'] = df[['so2', 'no2', 'rspm', 'spm']].mean(axis=1)

# Group data by state and calculate the mean of average pollution for each state
state_pollution = df.groupby('state')['average_pollution'].mean().reset_index()

# Sort states based on average pollution in descending order
top_5_states = state_pollution.sort_values(by='average_pollution', ascending=False).head(5)

print(top_5_states)

            state  average_pollution
8           Delhi         141.184127
34    Uttaranchal         118.212218
32  Uttar Pradesh         114.079715
3           Bihar         105.793081
33    Uttarakhand         104.749858


### Based on the provided data, it appears that Delhi has the highest average pollution level among all the states, with an average pollution level of approximately 141.18. Uttaranchal, Uttar Pradesh, Bihar, and Uttarakhand follow with the next highest average pollution levels. These findings suggest that Delhi is experiencing a higher level of pollution compared to the other states in the dataset.